In [116]:
import pennylane as qml
from pennylane import numpy as np
#from pennylane.optimize import NesterovMomentumOptimizer

#100 qubits required!!!!!!!!

dev = qml.device("default.qubit")
def layer(layer_weights):
    for wire in range(100):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in range(99):
        qml.CNOT([wires, wires+1])
    qml.CNOT([99,0])

def state_preparation(x):
    qml.BasisState(x, wires=list(range(100)))

@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))

In [117]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [56]:
# Convert binary strings to list of bits
def binary_to_bits(binary_list):
    return [[int(bit) for bit in binary_string] for binary_string in binary_list]
# Suppose we have the following binary dataset
X = np.array(binary_to_bits(['0110', '1001', '1101', '1111', '0000','0110', '1001', '1101', '1111', '0000']))
Y = np.array([0, 1, 1, 0, 1,0, 1, 1, 0, 1]) 
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")

x = [0 1 1 0], y = -1
x = [1 0 0 1], y = 1
x = [1 1 0 1], y = 1
x = [1 1 1 1], y = -1
x = [0 0 0 0], y = 1
x = [0 1 1 0], y = -1
x = [1 0 0 1], y = 1
x = [1 1 0 1], y = 1
x = [1 1 1 1], y = -1
x = [0 0 0 0], y = 1


In [57]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights:", weights_init)
print("Bias: ", bias_init)

Weights: [[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]
Bias:  0.0


In [59]:
#opt = NesterovMomentumOptimizer(0.5)
opt = qml.AdamOptimizer()
batch_size = 5
weights = weights_init
bias = bias_init
for it in range(10):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 0.8080190 | Accuracy: 0.8000000
Iter:    2 | Cost: 0.8016783 | Accuracy: 0.8000000
Iter:    3 | Cost: 0.7966312 | Accuracy: 0.8000000
Iter:    4 | Cost: 0.7902457 | Accuracy: 0.8000000
Iter:    5 | Cost: 0.7832047 | Accuracy: 0.8000000
Iter:    6 | Cost: 0.7771038 | Accuracy: 0.8000000
Iter:    7 | Cost: 0.7702430 | Accuracy: 0.8000000
Iter:    8 | Cost: 0.7627502 | Accuracy: 0.8000000
Iter:    9 | Cost: 0.7547155 | Accuracy: 0.8000000
Iter:   10 | Cost: 0.7462006 | Accuracy: 0.8000000
Iter:   11 | Cost: 0.7372791 | Accuracy: 0.8000000
Iter:   12 | Cost: 0.7280708 | Accuracy: 0.8000000
Iter:   13 | Cost: 0.7186356 | Accuracy: 0.8000000
Iter:   14 | Cost: 0.7089561 | Accuracy: 0.8000000
Iter:   15 | Cost: 0.6990257 | Accuracy: 0.8000000
Iter:   16 | Cost: 0.6891138 | Accuracy: 0.8000000
Iter:   17 | Cost: 0.6791942 | Accuracy: 0.8000000
Iter:   18 | Cost: 0.6692663 | Accuracy: 0.8000000
Iter:   19 | Cost: 0.6603268 | Accuracy: 0.8000000
Iter:   20 | Cost: 0.6514383 | 

In [60]:
data = np.loadtxt("AI_2qubits_training_data.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])

In [160]:
def binary_to_bits(binary_list):
    return [[[int(i) for i in str(bit)] for bit in binary_string][0] for binary_string in binary_list]
X = np.array(binary_to_bits(np.loadtxt("AI_2qubits_training_data.txt", dtype = str)[:,:-1]))
Y = np.loadtxt("AI_2qubits_training_data.txt", dtype = str)[:,-1].astype(int)
Y-=1
def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

# Convert labels to one-hot encoding
y_data_one_hot = one_hot(Y, 3)

In [153]:
np.random.seed(0)
num_qubits = 100
num_layers = 2
weights_init = 0.01 * np.random.randn(3, num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = qml.AdamOptimizer()
batch_size = 5
weights = weights_init
bias = bias_init
for it in range(10):
    for j in range(3):
        # Update the weights by one optimizer step, using only a limited batch of data
        batch_index = np.random.randint(0, len(X), (batch_size,))
        X_batch = np.array(X)[batch_index]
        Y_batch = y_data_one_hot[:,j][batch_index]
        weights, bias = opt.step(cost, weights[j], bias, X=X_batch, Y=Y_batch)
    
        # Compute accuracy
        predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
        current_cost = cost(weights, bias, X, Y)
        acc = accuracy(Y, predictions)
    
        print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

ValueError: maximum supported dimension for an ndarray is 32, found 100